In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1480.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  361.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4353 personas en los últimos 7 días 

Un positivo PCR cada 1774 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] != 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1480.0,235.0,422.0,510743.0,46.011399,82.624725,38.0
Condado-Campiña,486.0,111.0,172.0,155057.0,71.586578,110.926949,21.0
Huelva-Costa,760.0,85.0,192.0,288115.0,29.502109,66.640057,14.0
Huelva (capital),361.0,33.0,81.0,143663.0,22.970424,56.381949,6.0
Lepe,105.0,22.0,37.0,27431.0,80.201232,134.883890,6.0
Palma del Condado (La),69.0,31.0,37.0,10761.0,288.077316,343.834216,5.0
Gibraleón,25.0,9.0,11.0,12607.0,71.388911,87.253113,4.0
Manzanilla,16.0,11.0,12.0,2135.0,515.222482,562.060890,3.0
Sierra de Huelva-Andévalo Central,210.0,37.0,55.0,67571.0,54.757218,81.395865,3.0


In [14]:
if tabla[tabla['Diferencia PCR'] != 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Campillo (El),26.0,24.0,24.0,2023.0,1186.356896,1186.356896,0.0
Manzanilla,16.0,11.0,12.0,2135.0,515.222482,562.060890,3.0
Almendro (El),5.0,1.0,4.0,826.0,121.065375,484.261501,1.0
Valdelarco,2.0,1.0,1.0,239.0,418.410042,418.410042,NaN
Palma del Condado (La),69.0,31.0,37.0,10761.0,288.077316,343.834216,5.0
Bollullos Par del Condado,84.0,27.0,48.0,14272.0,189.181614,336.322870,0.0
Granado (El),2.0,1.0,1.0,516.0,193.798450,193.798450,0.0
Jabugo,5.0,1.0,4.0,2250.0,44.444444,177.777778,0.0
Villanueva de los Castillejos,7.0,5.0,5.0,2820.0,177.304965,177.304965,1.0
